# 🎬 YTautoma - YouTube Shorts Automation

Generate 60-second YouTube Shorts using local AI models.

| Component | Model |
|-----------|-------|
| Story | Gemma 3 (Ollama) |
| Images | Z-Image-Turbo |
| Video | Wan 2.2 |
| Voice | VibeVoice / Edge-TTS |

**Works on**: RunPod, Colab (A100), Lambda Labs

## 1️⃣ System Setup

In [1]:
# Install system dependencies
!apt-get update -qq && apt-get install -y -qq ffmpeg
!ffmpeg -version | head -1

W: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers


In [2]:
# Upgrade PyTorch (run this, then RESTART RUNTIME before next cell)
!pip install -q --upgrade typing_extensions torch torchvision
print('✅ Done! Now restart the runtime (Runtime > Restart runtime) and run the next cell')

✅ Done! Now restart the runtime (Runtime > Restart runtime) and run the next cell


In [1]:
# Verify PyTorch (run after restart)
import torch
print(f'PyTorch {torch.__version__} | CUDA: {torch.cuda.is_available()}')

PyTorch 2.9.1+cu128 | CUDA: True


In [2]:
!pip uninstall flash-attn -y && pip install "https://github.com/mjun0812/flash-attention-prebuild-wheels/releases/download/v0.6.8/flash_attn-2.7.4%2Bcu128torch2.9-cp312-cp312-linux_x86_64.whl"

Found existing installation: flash_attn 2.7.4+cu128torch2.9
Uninstalling flash_attn-2.7.4+cu128torch2.9:
  Successfully uninstalled flash_attn-2.7.4+cu128torch2.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.1/406.1 MB 227.5 MB/s  0:00:0100:0100:01


## 2️⃣ Clone & Install

In [3]:
import os

# Auto-detect workspace
WORKSPACE = '/content' if os.path.exists('/content') else '/workspace' if os.path.exists('/workspace') else os.path.expanduser('~')
os.chdir(WORKSPACE)

# Clone repo
!git clone https://github.com/DragonLord1998/YTautoma.git 2>/dev/null || (cd YTautoma && git pull)
os.chdir('YTautoma')
PROJECT_DIR = os.getcwd()
print(f'Project: {PROJECT_DIR}')

Already up to date.
Project: /workspace/YTautoma


In [4]:
# Install Python dependencies
!pip install  -r requirements.txt
!pip install  git+https://github.com/huggingface/diffusers
!pip install  edge-tts easydict

  Using cached https://github.com/mjun0812/flash-attention-prebuild-wheels/releases/download/v0.6.8/flash_attn-2.7.4%2Bcu128torch2.9-cp312-cp312-linux_x86_64.whl (406.1 MB)
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-zp6mt92r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-zp6mt92r
  Resolved https://github.com/huggingface/diffusers to commit 17c0e79dbdf53fb6705e9c09cc1a854b84c39249
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
# Install Ollama + Gemma 3
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess, time
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

!ollama pull gemma3:4b

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling aeda25e63ebd: 100% ▕██████████████████▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling b6ae5839783f: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
suc

In [ ]:
# Setup Wan 2.2 (video generation)
!mkdir -p models
!git clone https://github.com/Wan-Video/Wan2.2.git models/Wan2.2 2>/dev/null || echo 'Already cloned'
!pip install  -r models/Wan2.2/requirements.txt
!pip install hf_transfer
# Download model (10GB) - uncomment to enable video generation
!huggingface-cli download Wan-AI/Wan2.2-TI2V-5B --local-dir models/Wan2.2-TI2V-5B

Already cloned


In [ ]:
# Setup VibeVoice (high-quality TTS) - optional
!git clone https://github.com/microsoft/VibeVoice.git models/VibeVoice 2>/dev/null || echo 'Already cloned'
!cd models/VibeVoice && pip install  -e .

In [ ]:
# Create .env configuration
import os
PROJECT_DIR = os.getcwd()

env_content = f"""OLLAMA_MODEL=gemma3:4b
OLLAMA_BASE_URL=http://localhost:11434

ZIMAGE_MODEL=Tongyi-MAI/Z-Image-Turbo
ZIMAGE_DEVICE=cuda

WAN_REPO_PATH={PROJECT_DIR}/models/Wan2.2
WAN_MODEL_PATH={PROJECT_DIR}/models/Wan2.2-TI2V-5B
WAN_T5_CPU=true
WAN_OFFLOAD_MODEL=true

TTS_ENGINE=edge
VIBEVOICE_REPO_PATH={PROJECT_DIR}/models/VibeVoice

LOW_VRAM_MODE=true
TORCH_DTYPE=float16
"""

with open('.env', 'w') as f:
    f.write(env_content)

print('✅ Configuration saved!')

## 3️⃣ Generate YouTube Short

In [ ]:
# Quick test: Story only
#!python main.py --story-only -c mystery

In [ ]:
# Generate story + images
#!python main.py --images-only -c horror

In [ ]:
#Do not forget to rename Wan2.2_VAE.pth to Wan2.1_VAE.pth

In [ ]:
!pip install openai-whisper HyperPyYAML onnxruntime inflect wetext omegaconf conformer hydra-core lightning rich gdown matplotlib wget pyarrow pyworld librosa decord modelscope GitPython decord peft onnxruntime pandas matplotlib -e  git+https://github.com/facebookresearch/sam2.git@0e78a118995e66bb27d78518c4bd9a3e95b4e266#egg=SAM-2  loguru sentencepiece

In [ ]:
!pip install torch torchvision torchaudio opencv-python diffusers transformers tokenizers accelerate tqdm imageio[ffmpeg] easydict ftfy dashscope 

In [ ]:
!pip install --upgrade transformers peft diffusers

In [ ]:
# Full pipeline (requires Wan 2.2 model download)
!pip install ftfy
!pip install decord
!pip install librosa
!pip install opencv-python-headless

!python main.py -c sci-fi

## 4️⃣ View & Download

In [ ]:
# View generated images
from IPython.display import Image, display
import glob

images = sorted(glob.glob('output/**/base_image.png', recursive=True))[-6:]
for img in images:
    print(img.split('/')[-2])
    display(Image(filename=img, width=250))

In [ ]:
# Download outputs
import os, glob

try:
    from google.colab import files
    !cd output && zip -r ../output.zip .
    files.download('output.zip')
except ImportError:
    print('Find outputs at: ./output/')
    !find output -name '*.mp4' -o -name '*.png' | head -20